In [5]:
from classes.Dataset_Basic import Dataset_Basic
from classes.Process_Stages import Process_Stages

from classes.Dataset_Settings import Dataset_Settings

from run_models.cosine_sililarity.classes.Dataset_Cosine import Dataset_Cosine
from run_models.gensim.classes.Process_Stages_Gensim import Process_Stages_Gensim

from run_models.embed_words.classes.Embed_Words import Embed_Words

from run_models.gensim.services.gensim_services import gensim_download, gensim_save, gensim_load
from word_embedding.models.classes.EmbeddingModel import EmbeddingModel

name_dataset = "conceptnet_cosine_similarity"

datasets = {}
datasets["neural_course"] = Dataset_Cosine(
    df_name="neural_course",
    model_name=name_dataset, # used for dir name inside data_saved

    language="english",

    datasets = {
        "emebed_words_conceptnet": Dataset_Settings(
            df=None,
            may_run_now=False,
            required=True,
            parquet=True
        ),
        name_dataset: Dataset_Settings(
            df=None,
            may_run_now=True,
            required=True,
            parquet=True,
            name_required_dataset="gensim"
        ),
    },

    columns_to_add = {name_dataset: {"cosine_similarity": []}},

)

# get dataset, process dataset, save dataset
datasets["neural_course"].get_dataset()
datasets["neural_course"].process_dataset()
datasets["neural_course"].add_columns()
datasets["neural_course"].save()


Found neural_course with type .parquet in (data_saved/emebed_words_conceptnet) and converted it to df
df found: True, name: emebed_words_conceptnet
File neural_course not found in (data_saved/conceptnet_cosine_similarity)
df found: False, name: conceptnet_cosine_similarity
No need to run emebed_words_conceptnet for neural_course
Running conceptnet_cosine_similarity for neural_course
basic processing starting for neural_course


  0%|          | 0/646 [00:00<?, ?it/s]/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 646/646 [00:00<00:00, 30564.94it/s]

no saving needed because basic emebed_words_conceptnet already done on neural_course
saving new conceptnet_cosine_similarity phase for: neural_course


In [2]:
vars(datasets["neural_course"])
datasets["neural_course"]["datasets"]["cosine_similarity"]["df"]
# datasets["neural_course"]["add_columns"]


,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain,dataset_name,cosine_similarity
0,0,"Give a definition for the term ""artificial ne...",222,"[4.2985999644733965, 0.039699986577034, 1.1677...","[3.899799990002066, -0.36880000215023756, 0.82...",2,2,neural_networks,neural_course,0.914035
1,1,"Give a definition for the term ""artificial ne...",222,"[4.053999971598387, 0.18999999947845936, 0.941...","[3.899799990002066, -0.36880000215023756, 0.82...",2,2,neural_networks,neural_course,0.945509
2,2,"Give a definition for the term ""artificial ne...",222,"[1.6472999975085258, 0.019599996972829103, 0.3...","[3.899799990002066, -0.36880000215023756, 0.82...",1,2,neural_networks,neural_course,0.829610
3,3,"Give a definition for the term ""artificial ne...",222,"[5.877799991518259, -0.15270000789314508, 0.97...","[3.899799990002066, -0.36880000215023756, 0.82...",2,2,neural_networks,neural_course,0.680436
4,4,"Give a definition for the term ""artificial ne...",222,"[5.099799960851669, -0.5166999716311693, 1.214...","[3.899799990002066, -0.36880000215023756, 0.82...",2,2,neural_networks,neural_course,0.852299
...,...,...,...,...,...,...,...,...,...,...
641,641,Explain the Bias Variance Dilemma!,238,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.688400000333786, 1.6212000101804733, -0.085...",0,2,neural_networks,neural_course,1.000000
642,642,Explain the Bias Variance Dilemma!,238,"[1.259500004351139, 0.3689999971538782, 0.2881...","[4.688400000333786, 1.6212000101804733, -0.085...",0,2,neural_networks,neural_course,0.444338
643,643,Explain the Bias Variance Dilemma!,238,"[2.4142000041902065, 0.7708999924361706, -0.26...","[4.688400000333786, 1.6212000101804733, -0.085...",1,2,neural_networks,neural_course,0.752437
644,644,Explain the Bias Variance Dilemma!,238,"[3.38510000705719, 1.6390000013634562, 0.14189...","[4.688400000333786, 1.6212000101804733, -0.085...",1,2,neural_networks,neural_course,0.691208


In [2]:
import string
text = "hel. hwoe, that awesoem!"
text = text.translate(str.maketrans('', '', string.punctuation))
text

'hel hwoe that awesoem'

In [9]:

model = EmbeddingModel(
            model_name="conceptnet",
            download_link="conceptnet-numberbatch-17-06-300",
            download_func=gensim_download,
            save_func=gensim_save,
            load_func=gensim_load,
            dir_in_model_embeddin="/c/en/aag"
        )
model.load_model()


TypeError: __init__() got an unexpected keyword argument 'dir_in_model_embeddin'

In [7]:
# model.model.index_to_key

In [8]:
# model.model[f"/c/en/open"]